## 1.0 Getting Data

In [1]:
# importing useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import re

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Downloading the data from our github repository

In [2]:
url1 = 'https://raw.github.com/HamoyeHQ/stage-f-06-wine-tasting/master/data/top_40_varieties.zip'
url2 = 'https://raw.github.com/HamoyeHQ/stage-f-06-wine-tasting/master/data/top_varieties_count.csv'

In [3]:
top_40_varieties = pd.read_csv(url1)
top_varieties_count = pd.read_csv(url2)

In [4]:
# replacing every occurence of 'US' in country with 'United States of America'
top_40_varieties['country'].replace('US', 'United States of America', inplace=True)

# replacing every occurence of 'US' in not_vintage with 'United States of America'
top_40_varieties['not_vintage'] = top_40_varieties['not_vintage'].apply(lambda x: x.replace(\
                                                                        'US', 'United States of America'))

In [5]:
# renaming the columns in top_varieties_count
top_varieties_count = top_varieties_count.rename(columns={'variety': 'count', 'Unnamed: 0': 'variety'})
top_varieties_count = top_varieties_count.set_index('variety') # setting the index
top_varieties_count = top_varieties_count['count'] # making it a Series

In [6]:
top = 20 # selecting top 20 varities as our working varieties. note 1 < n <= 40

# making a datframe of our selecting top n varieties
top_df = top_40_varieties[top_40_varieties['variety'].isin(top_varieties_count.iloc[:top].index)]

In [7]:
# threshold of miniority variety to over sample (use sentences as document instead of the whole description)
minority_threshold = 5000 

# making a dataframe of the miniority classes
minority_df = top_df[top_df['variety'].isin(top_varieties_count[top_varieties_count < \
                                                                      minority_threshold].index)]

In [8]:
from nltk.tokenize import sent_tokenize # importing useful library

In [9]:
oversampled_miniority_lst = [] # empty list to store sentences as tokens miniority corpus

# creating a function to use sentences as tokens for the miniority classes
def over_sample_miniority(row):
    doc_list = sent_tokenize(row['description'])
    for sent in doc_list:
        row['description'] = sent
        oversampled_miniority_lst.append(list(row))

In [10]:
minority_df.apply(over_sample_miniority, axis=1); # over sample the miniority classes

In [11]:
# converts oversampled_miniority_lst to a dataframe
oversampled_miniority_df = pd.DataFrame(oversampled_miniority_lst, columns=minority_df.columns)

In [12]:
# selecting majority classes as a dataframe to concatenate to oversampled_miniority_lst
majority_df = top_df[top_df['variety'].isin(\
                                top_varieties_count[top_varieties_count >= minority_threshold].index)]

# concatenates majority_df to oversampled_miniority_lst
balanced_df = pd.concat([majority_df, oversampled_miniority_df]) 
balanced_df = balanced_df.reset_index().drop('index', axis=1) # resets index

In [13]:
balanced_variety = balanced_df['variety'].value_counts() # gets a Series of the variety count in balanced_df

In [14]:
# importing useful libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [15]:
# for sentence oversampling
sent_oversample_corpus = [doc1 + ' ' + doc2 for doc1, doc2 in zip(\
                                                balanced_df['description'], balanced_df['not_vintage'])]

In [16]:
labels = [label for label in balanced_df['variety']]

## 2.0 Data Preprocessing

In [17]:
import spacy

In [18]:
# creating a spacy pipeline and disabling tagger, parser and ner to speed up tokenizer
nlp = spacy.load('en', disable=['tagger', 'parser', 'ner']) 

In [19]:
spacy_stop_words = spacy.lang.en.STOP_WORDS # getting spacy's stop-words

In [20]:
response = requests.get('https://raw.github.com/Yoast/YoastSEO.js/develop/src/config/stopwords.js')
yoast_stop_words = response.content.decode()

In [21]:
pattern = r'\[.+\]'
match = re.search(pattern, yoast_stop_words)
yoast_stop_words = set(match.group()[1:-1].replace('"', '').replace(',', '').split())
print(f'length of yoast_stop_words is {len(yoast_stop_words)}\n')
print(yoast_stop_words)

length of yoast_stop_words is 153

{'very', "i've", 'until', 'where', 'into', 'itself', "i'll", 'only', 'ought', 'were', 'them', 'has', 'herself', 'but', 'about', 'as', 'these', 'further', 'theirs', "that's", 'more', "he's", 'below', 'do', "there's", 'there', 'himself', 'yours', 'up', 'before', "you'll", 'between', 'with', "when's", "we've", 'myself', 'any', 'his', 'her', 'nor', 'yourselves', 'hers', 'she', 'to', 'all', 'under', 'have', 'in', 'other', "she'd", "why's", 'me', 'most', 'your', "she's", 'how', "he'll", 'which', 'a', 'down', "we'd", 'who', 'once', 'be', "it's", 'because', 'after', 'did', 'during', 'that', 'same', 'are', 'having', 'than', 'through', 'when', 'our', 'both', "what's", 'their', 'am', 'they', 'or', 'each', 'been', 'again', 'you', "she'll", 'whom', "here's", 'yourself', 'if', "you're", 'few', 'out', 'while', "we'll", 'the', 'would', "i'd", "you've", "i'm", "where's", 'doing', 'him', 'above', 'was', 'he', 'being', 'and', 'against', 'for', 'i', 'is', 'so', 'could', 

In [22]:
stop_words_lemma = {word.lemma_.lower() for word in nlp(' '.join(spacy_stop_words | yoast_stop_words))} | \
{'-pron-', '10', '12', 'aah', 'aa', 'ab', 'aaa', 'aand', '16', '2', '20', '30', '4', '40', '5', '6', '7', \
 '8', '9'}

### creating custom transformers to encapsulate our preprocessing

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

In [24]:
class GetTokens(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words=stop_words_lemma):
        self.stop_words = stop_words
    
    # defining tokenzer function to tokenize the lower case lemma of documents in a corpus and 
    # filter out stop-words  
    def tokenize(self, text):
        return [word.lemma_.lower() for word in nlp(text) if word.is_alpha and word.lemma_.lower() \
                not in self.stop_words]

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.tokens = [self.tokenize(doc) for doc in X]
            
        return self.tokens

In [25]:
tokens = GetTokens()

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [27]:
class Text2Sequence(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.sequence_tokenizer = Tokenizer(oov_token='<oov>')

    def fit(self, X, y=None):
        self.sequence_tokenizer.fit_on_texts(X)
        self.words_indices = self.sequence_tokenizer.word_index
        return self
    
    def transform(self, X):
        self.get_sequences = self.sequence_tokenizer.texts_to_sequences(X)
        return self.get_sequences

In [28]:
text_2_seq = Text2Sequence()

In [29]:
from keras.preprocessing.sequence import pad_sequences

In [30]:
class Padding(BaseEstimator, TransformerMixin):
    def __init__(self, pad='post'):
        self.pad = pad
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.get_paddings = pad_sequences(X, padding=self.pad)
        return self.get_paddings

In [31]:
pad = Padding()

In [32]:
from sklearn.pipeline import Pipeline

In [33]:
data_prep_pipe = Pipeline([('get_tokens', tokens), ('text_2_sequence', text_2_seq), ('padding', pad)], verbose=1)

#### The above should return preprocessed X_train and y_train from data_prep_pipe.fit_transform(X_train, y_train)

## Training

In [34]:
from gensim.models import Word2Vec # importing Word2Vec

In [35]:
def get_embedding_matrix(model, word_index):
    vocab_size = len(word_index) + 1
    embedding_dim = model.wv.vector_size
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    for word in model.wv.vocab:
        ind = word_index[word]
        embedding_matrix[ind] = model[word]
        
    return embedding_matrix

In [36]:
from sklearn.utils import class_weight

In [37]:
# importing deep learning libraries
from tensorflow.keras.layers import Embedding, Dense, GlobalMaxPool1D, Conv1D, Dropout
from tensorflow.keras.models import Sequential

In [38]:
def multi_class_fbeta(ytrue , ypred, beta=1, weighted=True, raw=False, epsilon=1e-7):
    beta_squared = beta**2

    ytrue = tf.cast(ytrue, tf.float32)
    ypred= tf.cast(ypred, tf.float32)
    
    max_prob = tf.reduce_max(ypred, axis=-1, keepdims=True)
    ypred = tf.cast(tf.equal(ypred, max_prob), tf.float32)
        
    tp = tf.reduce_sum(ytrue*ypred, axis=0)
    predicted_positive = tf.reduce_sum(ypred, axis=0)
    actual_positive = tf.reduce_sum(ytrue, axis=0)
    
    precision = tp/(predicted_positive+epsilon)
    recall = tp/(actual_positive+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision + recall + epsilon)
    
    if raw:
        return fb
    
    if weighted:
        supports = tf.reduce_sum(ytrue, axis=0)
        return tf.reduce_sum(fb*supports / tf.reduce_sum(supports))

    return tf.reduce_mean(fb)

In [39]:
def build_cnn_model(embedding_matrix, input_length):
    model = Sequential()
    model.add(Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], 
                           weights=[embedding_matrix], 
                           input_length=input_length,
                           mask_zero=True,
                           trainable=False))
    
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(Conv1D(128, 3, activation='relu'))
    
    model.add(GlobalMaxPool1D())
    
    model.add(Dropout(0.2))
    
    model.add(Dense(20, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', multi_class_fbeta])

    return model

In [40]:
class CNNModel(BaseEstimator, ClassifierMixin):
    def __init__(self, epochs=7, batch_size=128, verbose=1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        
    def fit(self, X, y):
        self.corpus = data_prep_pipe.named_steps['get_tokens'].tokens
        self.w2v_model = Word2Vec(self.corpus, size=300, min_count=1, iter=10)
        self.embedding_matrix = get_embedding_matrix(self.w2v_model, \
                                                     data_prep_pipe.named_steps['text_2_sequence'].words_indices)
        
        self.model = build_cnn_model(self.embedding_matrix, X.shape[1])
        
        self.le = LabelEncoder()
        self.one_hot = OneHotEncoder(sparse=False) # initializes a LabelEncoder object
        
        encoded_labels = self.le.fit_transform(y)
        
        self.class_weights = class_weight.compute_class_weight('balanced', np.arange(20), encoded_labels)
        self.class_weights = dict(enumerate(self.class_weights))

        one_hot_labels = self.one_hot.fit_transform(encoded_labels.reshape(-1, 1))
        
        self.history = self.model.fit(X, one_hot_labels, epochs=self.epochs, batch_size=self.batch_size, \
                                      class_weight=self.class_weights, verbose=self.verbose)
        
        return self
    
    def predict(self, X):
        self.pred = self.model.predict(X)
        return self.pred 
        

In [41]:
cnn_model = CNNModel()

#### The above should be fitted on the preprocessed X_train and y_train from data_prep_pipe.fit_transform(X_train, y_train).

## Prediction

#### the user's input to be predicted will be transformed by being fed into the preprocessor by actually doing somehing like prep_input = data_prep_pipe.transform(user_input) and then be fed into the model by doing something like pred = cnn_model.predict(prep_input)

# Note: lines of code below are for demostration purposes and should not be included in our deployment.

### Our model only predicts the top 20 varieties and they are as listed below

In [42]:
balanced_df['variety'].unique()

array(['Riesling', 'Pinot Noir', 'Cabernet Sauvignon', 'Chardonnay',
       'Red Blend', 'Bordeaux-style Red Blend', 'White Blend',
       'Portuguese Red', 'Pinot Gris', 'Malbec', 'Merlot',
       'Sauvignon Blanc', 'Sangiovese', 'Rosé', 'Zinfandel', 'Syrah',
       'Nebbiolo', 'Rhône-style Red Blend', 'Sparkling Blend',
       'Tempranillo'], dtype=object)

In [43]:
balanced_df['variety'].value_counts() # checking for miniority and majority classes

Pinot Noir                  13272
Sauvignon Blanc             12801
Chardonnay                  11753
Syrah                       11128
Cabernet Sauvignon           9472
Rosé                         9207
Red Blend                    8946
Nebbiolo                     8107
Malbec                       8047
Merlot                       7874
Portuguese Red               7734
Sangiovese                   7432
Bordeaux-style Red Blend     6915
Zinfandel                    6512
White Blend                  6034
Sparkling Blend              5740
Tempranillo                  5552
Riesling                     5189
Rhône-style Red Blend        3972
Pinot Gris                   3961
Name: variety, dtype: int64

In [44]:
model_pipe = Pipeline([('data_prep', data_prep_pipe), ('model', cnn_model)])

In [45]:
model_pipe.fit(sent_oversample_corpus, labels)

[Pipeline] ........ (step 1 of 3) Processing get_tokens, total= 1.1min
[Pipeline] ... (step 2 of 3) Processing text_2_sequence, total=   6.9s
[Pipeline] ........... (step 3 of 3) Processing padding, total=   1.8s
Epoch 1/7
1248/1248 [==============================] - 11s 9ms/step - loss: 0.8241 - accuracy: 0.7354 - multi_class_fbeta: 0.7361
Epoch 2/7
1248/1248 [==============================] - 10s 8ms/step - loss: 0.5940 - accuracy: 0.8010 - multi_class_fbeta: 0.8021
Epoch 3/7
1248/1248 [==============================] - 11s 8ms/step - loss: 0.5375 - accuracy: 0.8166 - multi_class_fbeta: 0.8178
Epoch 4/7
1248/1248 [==============================] - 11s 9ms/step - loss: 0.4980 - accuracy: 0.8287 - multi_class_fbeta: 0.8296
Epoch 5/7
1248/1248 [==============================] - 10s 8ms/step - loss: 0.4709 - accuracy: 0.8365 - multi_class_fbeta: 0.8376
Epoch 6/7
1248/1248 [==============================] - 10s 8ms/step - loss: 0.4485 - accuracy: 0.8432 - multi_class_fbeta: 0.8442
Epoch 7

Pipeline(steps=[('data_prep',
                 Pipeline(steps=[('get_tokens', GetTokens()),
                                 ('text_2_sequence', Text2Sequence()),
                                 ('padding', Padding())],
                          verbose=1)),
                ('model', CNNModel())])

### Let's do a demo prediction of a majority class'Pinot Noir' wine variety from a recent review (published 12/1/2020) not in our training set from [wine ethusiast](https://www.winemag.com/buying-guide/albert-bichot-2018-domaine-du-pavilion-clos-des-marechaudes-grand-cru-corton/). As we know, adding info like country, designation, province, region1, region2, and winery as the last sentence of our wine description boosted our model's performance during training. We therefore advise users to input them also for best performance.

In [46]:
user_input = "A hint of woodsmoke combines with this wine's shy aromas of black cherry. " + \
              "The palate comes in with surprising grace and svelteness, presenting a filigreed but " + \
              "firm structure. Cherry aromas shimmering with Morello and Amarena are expressed against " + \
              "that frame, crunching slightly and heightened by vivid freshness. This wine has compact, " + \
              "elegant power and inherent conviction. Drink through 2050. France Domaine du Pavilion Clos " + \
              "des Maréchaudes Grand Cru Burgundy Corton Albert Bichot"

### Notice the double quotes we used. This is to prevent error should the user's input contain apostrophe.

In [47]:
user_input

"A hint of woodsmoke combines with this wine's shy aromas of black cherry. The palate comes in with surprising grace and svelteness, presenting a filigreed but firm structure. Cherry aromas shimmering with Morello and Amarena are expressed against that frame, crunching slightly and heightened by vivid freshness. This wine has compact, elegant power and inherent conviction. Drink through 2050. France Domaine du Pavilion Clos des Maréchaudes Grand Cru Burgundy Corton Albert Bichot"

In [48]:
# predicting user's description
def get_prediction(user_inp):
    pred = model_pipe.predict([user_inp])
    top_5_pred = np.argsort(pred[0])[-1:-6:-1]
    
    print("The wine's variety is most likely {} with a probability of {}%".format(\
                                            model_pipe.named_steps['model'].le.inverse_transform(\
                                            [top_5_pred[0]])[0], round(pred[0][top_5_pred[0]]*100, 3)))
        
    print('\nOther possible varieties are:\n')

    for i in range(1, len(top_5_pred)):
        print('{} ==> {}%'.format(model_pipe.named_steps['model'].le.inverse_transform(\
                                [top_5_pred[i]])[0], round(pred[0][top_5_pred[i]]*100, 3)))

In [49]:
get_prediction(user_input)

The wine's variety is most likely Pinot Noir with a probability of 99.998%

Other possible varieties are:

Chardonnay ==> 0.002%
Red Blend ==> 0.0%
Cabernet Sauvignon ==> 0.0%
Bordeaux-style Red Blend ==> 0.0%


### ...and voila, our model's prediction was as good as right!

### Let's consider predicting a miniority class 'Pinot Gris' wine variety from a recent review (published 12/1/2020) not in our training set from [wine ethusiast](https://www.winemag.com/buying-guide/antiquum-farm-2019-daisy-pinot-gris-willamette-valley/).

In [50]:
user_input2 = "This breaks new ground for Oregon's signature white grape. It's rich and leesy, fragrant " + \
              "and full, a bonanza of apple, pear and melon fruits. There's a lingering trail of honeysuckle, " + \
              "and a lush minerality that underscores the freshness. A stunning wine. United States of America " + \
              "Diasy Oregon Willamette Valley Willamette Valley Antiquum Farm"

In [51]:
user_input2

"This breaks new ground for Oregon's signature white grape. It's rich and leesy, fragrant and full, a bonanza of apple, pear and melon fruits. There's a lingering trail of honeysuckle, and a lush minerality that underscores the freshness. A stunning wine. United States of America Diasy Oregon Willamette Valley Willamette Valley Antiquum Farm"

In [52]:
get_prediction(user_input2)

The wine's variety is most likely Pinot Gris with a probability of 40.686%

Other possible varieties are:

Riesling ==> 26.947%
Chardonnay ==> 22.473%
White Blend ==> 9.554%
Sparkling Blend ==> 0.16%
